<center>Progetto realizzato da Elena Curti (matr. 185431)

# Crimes Database
</center>

## Introduzione
Lo scopo di questo progetto è di rispondere a dei research 

## Requisiti


## Operazioni iniziali
Creare un database ...

In [1]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687",  auth=("neo4j", "password1234"))
print("Connessione al database eseguita correttamente!")

Connessione al database eseguita correttamente!


## Descrizione del database

## 1. Quali sono gli agenti di polizia più importanti?


### 1A. Numero dei crimini (cypher query) 
1A. In base al numero dei crimini in cui hanno investigato (usando le cypher query) 

In [2]:
def print_n_results(cq, n=100):
    ris = graph.run(cq)
    ris.sample_size = n
    display(ris)

cq = """ 
MATCH (o:Officer)<-[:INVESTIGATED_BY]-(c:Crime) 
WITH o, count(c) as num_invest
RETURN o.badge_no as badge_number, o.name as nome, o.surname as cognome, o.rank as rango, num_invest
ORDER BY num_invest DESC
LIMIT 10 ;
"""
print_n_results(cq)


# TODO questo si può rimuovere, usarlo solo per capire come cambiano i risultati
ris1 = [i["badge_number"] + " " + i["nome"] +" " + i["cognome"] for i in graph.run(cq).data()] 
def check_results(cq):
    res = [i["badge_number"] + " " + i["nome"] +" " + i["cognome"] for i in graph.run(cq).data()] 
    return res == ris1



badge_number,nome,cognome,rango,num_invest
48-0216838,Madelon,DeSousa,Sergeant,50
04-5664884,Cloe,Ings,Police Constable,47
58-9758255,Kania,Notti,Sergeant,46
27-2227814,Worthy,Nettles,Inspector,45
86-7588000,Ricca,Miskimmon,Inspector,44
79-9843712,Olenolin,Klehyn,Sergeant,44
41-5548000,Winonah,Skynner,Inspector,44
36-7678091,Simmonds,Greensall,Police Constable,43
34-6222792,Hollyanne,Orcott,Inspector,43
73-9322216,Hedy,Vinson,Police Constable,43


### 1B. Numero dei crimini (Page Rank e Betweenness Centrality)

In [3]:
# Creo la proiezione
if graph.run("call gds.graph.exists('officers')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("officers")')

cq = """
CALL gds.graph.project(
  'officers',    
  ['Officer', 'Crime'],
  {INVESTIGATED_BY: {orientation: 'UNDIRECTED'} }
) ;
"""

graph.run(cq)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{Crime: {label: 'Crime', properties: {}}, Officer: {label: 'Officer', properties: {}}}","{INVESTIGATED_BY: {orientation: 'UNDIRECTED', aggregation: 'DEFAULT', type: 'INVESTIGATED_BY', properties: {}}}",officers,29762,57524,10


In [4]:
# Memory estimation
graph.run(""" 
CALL gds.pageRank.write.estimate('officers', { writeProperty: 'pageRank' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN "Page Rank" as algoritmo, nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory

UNION ALL

CALL gds.betweenness.write.estimate('officers', { writeProperty: 'betweenness' })
YIELD  nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN "Betweenness" as algoritmo, nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory ; 
""" )

algoritmo,nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
Page Rank,29762,57524,718808,718808,701 KiB
Betweenness,29762,57524,8335024,8335024,8139 KiB


In [5]:
# Risultati
print("Page Rank: ")
cq=""" 
CALL gds.pageRank.stream('officers')
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS n, score AS pageRank
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome, n.rank as rango,pageRank
ORDER BY pageRank DESC
LIMIT 10 ;
"""
print_n_results(cq)


print("Betweenness Centrality: ")
cq=""" 
CALL gds.betweenness.stream('officers')
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS n, score AS pageRank
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome, n.rank as rango,pageRank
ORDER BY pageRank DESC
LIMIT 10 ;
"""
print_n_results(cq)

Page Rank: 


badge_number,nome,cognome,rango,pageRank
48-0216838,Madelon,DeSousa,Sergeant,22.602140755580333
04-5664884,Cloe,Ings,Police Constable,21.27718767680494
58-9758255,Kania,Notti,Sergeant,20.835536650546473
27-2227814,Worthy,Nettles,Inspector,20.39388562428801
79-9843712,Olenolin,Klehyn,Sergeant,19.952234598029545
86-7588000,Ricca,Miskimmon,Inspector,19.952234598029545
41-5548000,Winonah,Skynner,Inspector,19.952234598029545
36-7678091,Simmonds,Greensall,Police Constable,19.510583571771075
34-6222792,Hollyanne,Orcott,Inspector,19.510583571771075
73-9322216,Hedy,Vinson,Police Constable,19.510583571771075


Betweenness Centrality: 


badge_number,nome,cognome,rango,pageRank
48-0216838,Madelon,DeSousa,Sergeant,1225.0
04-5664884,Cloe,Ings,Police Constable,1081.0
58-9758255,Kania,Notti,Sergeant,1035.0
27-2227814,Worthy,Nettles,Inspector,990.0
41-5548000,Winonah,Skynner,Inspector,946.0
86-7588000,Ricca,Miskimmon,Inspector,946.0
79-9843712,Olenolin,Klehyn,Sergeant,946.0
36-7678091,Simmonds,Greensall,Police Constable,903.0
34-6222792,Hollyanne,Orcott,Inspector,903.0
73-9322216,Hedy,Vinson,Police Constable,903.0


### 1C. Solo gli agenti che hanno arrestato almeno una persona (Degree centrality)

In [6]:
# Creo la proiezione
if graph.run("call gds.graph.exists('officers-crime-con-persone')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("officers-crime-con-persone")')

cq=""" 
CALL gds.graph.project.cypher('officers-crime-con-persone',
  'MATCH (o:Officer|Crime)  RETURN DISTINCT id(o) as id',
  'MATCH (o:Officer)<--(c:Crime)<--(:Person) RETURN  id(o) AS source, id(c) as target, "INVESTIGATED_BY" as type',
{validateRelationships:false}
);""" 

graph.run(cq)

nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,projectMillis
MATCH (o:Officer|Crime) RETURN DISTINCT id(o) as id,"MATCH (o:Officer)<--(c:Crime)<--(:Person) RETURN id(o) AS source, id(c) as target, ""INVESTIGATED_BY"" as type",officers-crime-con-persone,29762,55,46


In [7]:
# Memory estimation
graph.run(""" 
CALL gds.degree.write.estimate('officers-crime-con-persone', { writeProperty: 'betweenness' })
YIELD  nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory ; 
""" )

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
29762,55,56,56,56 Bytes


In [8]:
# Seleziono solo gli agenti che hanno arrestato una persona
cq=""" 
CALL gds.degree.stream('officers-crime-con-persone')
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS n, score
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome, n.rank as rango,score
ORDER BY score DESC
LIMIT 10 ;
"""
print_n_results(cq)

badge_number,nome,cognome,rango,score
26-5234182,Devy,Larive,Police Constable,3.0
27-0802784,Lilian,Dubbin,Sergeant,1.0
74-2393363,Paolina,Hellwig,Police Constable,1.0
69-8126297,Pauline,Petrasso,Police Constable,1.0
53-7043460,Edithe,Thackray,Inspector,1.0
30-2717379,Dorothea,Blann,Police Constable,1.0
04-6245275,Von,Death,Police Constable,1.0
54-9607307,Gregorius,Shakesby,Police Constable,1.0
80-2249206,Shepperd,Meller,Sergeant,1.0
92-0932678,Mada,Filisov,Police Constable,1.0


### 1D. In base al numero di persone arrestate (HITS)

In [9]:
# Creo la proiezione
if graph.run("call gds.graph.exists('officers-arresti')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("officers-arresti")')

cq=""" 
CALL gds.graph.project.cypher('officers-arresti',
  'MATCH (o:Officer|Person)  RETURN DISTINCT id(o) as id',
  'MATCH (o:Officer)<--(c:Crime)<--(p:Person) RETURN distinct id(p) as source, id(o) as target',
{validateRelationships:false}
);""" 

graph.run(cq)

nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,projectMillis
MATCH (o:Officer|Person) RETURN DISTINCT id(o) as id,"MATCH (o:Officer)<--(c:Crime)<--(p:Person) RETURN distinct id(p) as source, id(o) as target",officers-arresti,1369,54,9


In [10]:
# In base al numero di persone arrestate
cq=""" 
CALL gds.alpha.hits.stream('officers-arresti',  {hitsIterations: 1})
YIELD nodeId, values
WITH gds.util.asNode(nodeId) AS n, values as score
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome, n.rank as rango,score
ORDER BY score DESC
LIMIT 10 ;
"""
print_n_results(cq)

badge_number,nome,cognome,rango,score
26-5234182,Devy,Larive,Police Constable,"{hub: 0.0, auth: 0.2672612419124244}"
53-7043460,Edithe,Thackray,Inspector,"{hub: 0.0, auth: 0.1336306209562122}"
54-9607307,Gregorius,Shakesby,Police Constable,"{hub: 0.0, auth: 0.1336306209562122}"
80-2249206,Shepperd,Meller,Sergeant,"{hub: 0.0, auth: 0.1336306209562122}"
04-6245275,Von,Death,Police Constable,"{hub: 0.0, auth: 0.1336306209562122}"
30-2717379,Dorothea,Blann,Police Constable,"{hub: 0.0, auth: 0.1336306209562122}"
27-0802784,Lilian,Dubbin,Sergeant,"{hub: 0.0, auth: 0.1336306209562122}"
69-8126297,Pauline,Petrasso,Police Constable,"{hub: 0.0, auth: 0.1336306209562122}"
74-2393363,Paolina,Hellwig,Police Constable,"{hub: 0.0, auth: 0.1336306209562122}"
92-0932678,Mada,Filisov,Police Constable,"{hub: 0.0, auth: 0.1336306209562122}"


### 1E. Solo gli agenti di rango maggiore (Degree Centrality)

In [33]:
graph.run("MATCH (o:Officer) RETURN COLLECT(DISTINCT o.rank) AS tutti_i_ranghi")

tutti_i_ranghi
"['Chief Inspector', 'Inspector', 'Police Constable', 'Sergeant']"


In [11]:
if graph.run("call gds.graph.exists('officers-chief')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("officers-chief")')

cq = """ 
CALL gds.graph.project.cypher('officers-chief',
  'MATCH (o:Officer|Crime) WHERE o.rank="Chief Inspector" OR o.rank IS NULL RETURN DISTINCT id(o) as id',
  'MATCH (o:Officer)<--(c:Crime) RETURN  id(o) AS source, id(c) as target',
  {validateRelationships:false}
);
"""

graph.run(cq)

nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,projectMillis
"MATCH (o:Officer|Crime) WHERE o.rank=""Chief Inspector"" OR o.rank IS NULL RETURN DISTINCT id(o) as id","MATCH (o:Officer)<--(c:Crime) RETURN id(o) AS source, id(c) as target",officers-chief,28852,2500,31


In [12]:
# Memory estimation
graph.run(""" 
CALL gds.degree.write.estimate('officers-chief', { writeProperty: 'betweenness' })
YIELD  nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory ; 
""" )

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
28852,2500,56,56,56 Bytes


In [13]:
# Agenti di polizia con il rango maggiore (Chief Inspector) che hanno investigato in più crimini
cq=""" 
CALL gds.degree.stream('officers-chief')
YIELD nodeId, score 
WITH gds.util.asNode(nodeId) AS n, score 
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome,n.rank as rango, score
ORDER BY score DESC
LIMIT 10 ;
"""
print_n_results(cq)



badge_number,nome,cognome,rango,score
71-6459623,Kort,Monelli,Chief Inspector,40.0
14-3562764,Urban,Stave,Chief Inspector,39.0
49-5476687,Roberto,Febry,Chief Inspector,38.0
34-1935189,Esma,Gorrie,Chief Inspector,37.0
97-6686521,Whitney,Jeandot,Chief Inspector,37.0
75-9485265,Evey,Rahlof,Chief Inspector,37.0
19-8319581,Dottie,Syddie,Chief Inspector,37.0
45-7784686,Reinold,Fley,Chief Inspector,36.0
29-1687505,Kerwin,Gilluley,Chief Inspector,36.0
30-0611369,Helenka,Stutard,Chief Inspector,35.0


### Conclusioni
...

## 2. Trovare i gruppi di criminali

In [14]:

def print_in_columns(my_list):
    columns = 2  #if len(my_list)>10 else 3
    len_max = str(max([len(i) for i in my_list]) + 4)
    for i in range(0, len(my_list), columns):
        print("  ", end="")
        [eval('print(f"{i:'+len_max+'}", end="")') for i in my_list[i:i+columns]]
        print("")

def print_id_gruppo(query_results):
  for (communityId, gruppo) in query_results:
      print("---------- Gruppo "+str(communityId)+" ----------")
      # print(gruppo)
      print_in_columns(gruppo)
      print("")


### 2A. Con le conoscenze generiche (Algoritmo di Louvain)


In [15]:
## Creo la proiezione
NOME_GRAPH_CRIMINALS = "criminals-knows"
if graph.run("call gds.graph.exists('criminals-knows')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("criminals-knows")')

cq = """ 
  CALL gds.graph.project.cypher('criminals-knows',
  'MATCH (p:Person)-->(:Crime) RETURN id(p) as id',
  'MATCH (criminal:Person)-[:KNOWS]-(conoscente:Person) RETURN DISTINCT id(conoscente) as target, id(criminal) as source',
  {validateRelationships:false}
); 
"""
graph.run(cq)

nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,projectMillis
MATCH (p:Person)-->(:Crime) RETURN id(p) as id,"MATCH (criminal:Person)-[:KNOWS]-(conoscente:Person) RETURN DISTINCT id(conoscente) as target, id(criminal) as source",criminals-knows,29,30,24


In [16]:
graph.run("""CALL gds.louvain.write.estimate('criminals-knows', { writeProperty: 'community' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory ;
""")

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
29,30,7153,566272,[7153 Bytes ... 553 KiB]


In [17]:
cq = """ 
CALL gds.louvain.stream('criminals-knows')
  YIELD nodeId, communityId
WITH gds.util.asNode(nodeId) AS node, communityId
WITH communityId, apoc.coll.sort(collect(node.name + " " + node.surname + " (" + node.nhs_no+")")) AS gruppo
WHERE size(gruppo)>1  
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 27 ----------
  Diana Murray (900-41-3309)          Jessica Kelly (311-75-6483)         
  Kathleen Peters (250-75-5238)       Phillip Williamson (337-28-4424)    
  Raymond Walker (879-22-8665)        

---------- Gruppo 17 ----------
  Alan Ward (881-20-2396)        Brian Morales (335-71-7747)    
  Jack Powell (249-54-6589)      

---------- Gruppo 24 ----------
  Billy Moore (846-48-9238)         Lillian Martinez (397-28-4474)    

---------- Gruppo 25 ----------
  Andrea Montgomery (351-83-4608)    Donald Robinson (873-28-7561)      



### 2B. Dando un peso al tipo di conoscenze (Modularity Optimization)

In [18]:
## Creo il project
if graph.run("call gds.graph.exists('criminals-knows-pesate')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("criminals-knows-pesate")')

cq = """ 
  CALL gds.graph.project.cypher('criminals-knows-pesate',
   'MATCH (p:Person)-->(:Crime) RETURN id(p) as id',
   'MATCH (criminal:Person)-[]-(conoscente:Person)
    RETURN id(conoscente) as target,
    CASE
        WHEN EXISTS ( (conoscente)-[:KNOWS_LW]-(criminal) ) THEN 10
        WHEN EXISTS ( (conoscente)-[:FAMILY_REL]-(criminal) ) THEN 8
        WHEN EXISTS ( (conoscente)-[:KNOWS_PHONE]-(criminal) ) THEN 4
        WHEN EXISTS ( (conoscente)-[:KNOWS_SN]-(criminal) ) THEN 3
    END AS peso, id(criminal) as source',
  {validateRelationships:false}
); 
"""
graph.run(cq)

nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,projectMillis
MATCH (p:Person)-->(:Crime) RETURN id(p) as id,"MATCH (criminal:Person)-[]-(conoscente:Person) RETURN id(conoscente) as target, CASE WHEN EXISTS ( (conoscente)-[:KNOWS_LW]-(criminal) ) THEN 10 WHEN EXISTS ( (conoscente)-[:FAMILY_REL]-(criminal) ) THEN 8 WHEN EXISTS ( (conoscente)-[:KNOWS_PHONE]-(criminal) ) THEN 4 WHEN EXISTS ( (conoscente)-[:KNOWS_SN]-(criminal) ) THEN 3 END AS peso, id(criminal) as source",criminals-knows-pesate,29,60,30


In [19]:
cq = """ 
CALL gds.beta.modularityOptimization.stream('criminals-knows-pesate', {relationshipWeightProperty: 'peso' })
  YIELD nodeId, communityId
WITH gds.util.asNode(nodeId) AS node, communityId
WITH communityId, apoc.coll.sort(collect(node.name + " " + node.surname + " (" + node.nhs_no+")")) AS gruppo
WHERE size(gruppo)>1  
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 14 ----------
  Diana Murray (900-41-3309)       Kathleen Peters (250-75-5238)    
  Raymond Walker (879-22-8665)     

---------- Gruppo 17 ----------
  Alan Ward (881-20-2396)        Brian Morales (335-71-7747)    
  Jack Powell (249-54-6589)      

---------- Gruppo 24 ----------
  Billy Moore (846-48-9238)         Lillian Martinez (397-28-4474)    

---------- Gruppo 25 ----------
  Andrea Montgomery (351-83-4608)    Donald Robinson (873-28-7561)      

---------- Gruppo 28 ----------
  Jessica Kelly (311-75-6483)         Phillip Williamson (337-28-4424)    



### 2C. Includendo i conoscenti dei criminali (Strongly Connected Components)

In [20]:
## Creo il project
if graph.run("call gds.graph.exists('criminals-conoscenti')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("criminals-conoscenti")')

cq = """ 
  CALL gds.graph.project.cypher('criminals-conoscenti',
  ' MATCH (p:Person) 
    WHERE 
        EXISTS ( (p)-[:PARTY_TO]->(:Crime) )                    // Persone che hanno preso parte a un crimine
        OR EXISTS ( (p)-[]-(:Person)-[:PARTY_TO]->(:Crime) )   // Conoscenti dei criminali
    RETURN id(p) as id',
  
  'MATCH (criminal)-[:KNOWS]-(conoscente:Person) RETURN DISTINCT id(conoscente) as target, id(criminal) as source',
  {validateRelationships:false}
); 
"""
graph.run(cq)
print("Proiezione crata correttamente!")

Proiezione crata correttamente!


In [21]:
cq = """ 
CALL gds.alpha.scc.stream('criminals-conoscenti')
YIELD nodeId, componentId
WITH gds.util.asNode(nodeId) AS node, componentId
WITH componentId, apoc.coll.sort(collect(node.name + " " + node.surname + " (" + node.nhs_no+")")) AS gruppo
WHERE size(gruppo)>1  
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 0 ----------
  Alan Ward (881-20-2396)                Amanda Robertson (455-19-0708)         
  Amy Bailey (276-19-9235)               Amy Murphy (367-54-3328)               
  Andrea George (800-46-2184)            Andrea Montgomery (351-83-4608)        
  Andrea Moreno (240-77-5251)            Ann Fox (576-99-9244)                  
  Anna Chapman (878-32-2595)             Anne Freeman (804-54-6976)             
  Annie Duncan (863-96-9468)             Annie George (575-05-6519)             
  Arthur Willis (271-78-8919)            Ashley Robertson (554-93-4466)         
  Billy Moore (846-48-9238)              Bobby Russell (680-93-7668)            
  Bonnie Gilbert (622-53-3302)           Brandon Martin (853-69-5350)           
  Brenda Edwards (778-24-6852)           Brian Morales (335-71-7747)            
  Carl Fuller (358-70-5810)              Carl Lawrence (271-53-9609)            
  Carlos Black (859-81-0332)             Carlos Matthews (896-25-8370)        

### 2D. Aggiungo i "vicini" (Weakly Connected Components)

Come si può vedere dalla prossima query, esistono persone senza un legame di conoscenza che però abitano vicine. 

In [22]:
display(graph.run(""" 
MATCH (p1:Person)-->(:Location)-->(:Area)<--(:Location)<--(p2:Person) 
WHERE NOT EXISTS ( (p1)-[]-(p2))
RETURN COUNT(*) AS n """))


n
1800


Aggiungo quindi questa possibile conoscenza, tra le conoscenze già presenti, con un peso basso

In [23]:
# Creo il project
if graph.run("call gds.graph.exists('criminals-knows-pesate-con-vicini')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("criminals-knows-pesate-con-vicini")')

cq = """ 
  CALL gds.graph.project.cypher('criminals-knows-pesate-con-vicini',
   'MATCH (p:Person)-->(:Crime) RETURN id(p) as id',
   'MATCH (criminal:Person),(conoscente:Person)
    WHERE criminal<>conoscente
    RETURN id(conoscente) as target,
    CASE
        WHEN EXISTS ( (conoscente)-[:KNOWS_LW]-(criminal) ) THEN 10
        WHEN EXISTS ( (conoscente)-[:FAMILY_REL]-(criminal) ) THEN 8
        WHEN EXISTS ( (conoscente)-[:KNOWS_PHONE]-(criminal) ) THEN 4
        WHEN EXISTS ( (conoscente)-[:KNOWS_SN]-(criminal) ) THEN 3
        WHEN EXISTS ((criminal)-->(:Location)-->(:Area)<--(:Location)<--(conoscente) ) THEN 1
    END AS peso, id(criminal) as source',
  {validateRelationships:false}
); 
"""
graph.run(cq)

nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,projectMillis
MATCH (p:Person)-->(:Crime) RETURN id(p) as id,"MATCH (criminal:Person),(conoscente:Person) WHERE criminal<>conoscente RETURN id(conoscente) as target, CASE WHEN EXISTS ( (conoscente)-[:KNOWS_LW]-(criminal) ) THEN 10 WHEN EXISTS ( (conoscente)-[:FAMILY_REL]-(criminal) ) THEN 8 WHEN EXISTS ( (conoscente)-[:KNOWS_PHONE]-(criminal) ) THEN 4 WHEN EXISTS ( (conoscente)-[:KNOWS_SN]-(criminal) ) THEN 3 WHEN EXISTS ((criminal)-->(:Location)-->(:Area)<--(:Location)<--(conoscente) ) THEN 1 END AS peso, id(criminal) as source",criminals-knows-pesate-con-vicini,29,812,1660


In [24]:
graph.run(""" 
CALL gds.wcc.write.estimate('criminals-knows-pesate-con-vicini', { writeProperty: 'component' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
""")

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
29,812,328,328,328 Bytes


In [25]:
cq = """ 
CALL gds.wcc.stream('criminals-knows-pesate-con-vicini', { relationshipWeightProperty: 'peso' })
YIELD nodeId, componentId
WITH gds.util.asNode(nodeId) AS node, componentId
WITH componentId, apoc.coll.sort(collect(node.name + " " + node.surname + " (" + node.nhs_no+")")) AS gruppo
WHERE size(gruppo)>1  
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 0 ----------
  Alan Ward (881-20-2396)             Amanda Robertson (455-19-0708)      
  Amy Bailey (276-19-9235)            Andrea Montgomery (351-83-4608)     
  Annie George (575-05-6519)          Billy Moore (846-48-9238)           
  Brian Morales (335-71-7747)         Carlos Black (859-81-0332)          
  Craig Marshall (890-58-5813)        David Mills (589-69-0106)           
  Diana Murray (900-41-3309)          Donald Robinson (873-28-7561)       
  Ernest Clark (205-52-5697)          Fred Williamson (468-82-3915)       
  Gary Vasquez (679-81-5309)          Jack Powell (249-54-6589)           
  Jessica Kelly (311-75-6483)         Joan Flores (247-72-6304)           
  Kathleen Peters (250-75-5238)       Kenneth Carroll (252-29-4929)       
  Lillian Martinez (397-28-4474)      Maria Hughes (678-06-9352)          
  Michelle Patterson (600-92-0643)    Norma Payne (699-85-1673)           
  Phillip Williamson (337-28-4424)    Raymond Walker (879-22-8665)   

### Conclusioni
...

## 3. Shortest path tra persone con precedenti di spaccio
Fare lo shortest path per cercare possibili connessioni e raggruppare in base alla posizione geografica

Prendo due persone: una coinvolta in un crimine di tipo Drugs e l'altra pregiudicata ma che non abbia mai compiuto un crimine Drugs. Queste due persone inoltre non devono conoscersi direttamente:

In [26]:
cq = """ 
MATCH (p1:Person)-->(c1:Crime {type:"Drugs"}), (p2:Person)-->(c2:Crime)
WHERE 
    NOT EXISTS ( (p2)-->(:Crime {type:"Drugs"} )) 
    AND NOT EXISTS ( (p1)-[]-(p2)) 
    AND NOT EXISTS ((p1)-->(:Location)-->(:Area)<--(:Location)<--(p2) )
RETURN p1.nhs_no AS spacciatore_nhs_no, p1.name AS spacciatore_name, p2.nhs_no AS pregiudicato_nhs_no, p2.name AS pregiudicato_name
LIMIT 1
"""
display(graph.run(cq))


data = graph.run(cq).data()[0]
spacciatore_nhs_no = data["spacciatore_nhs_no"]
pregiudicato_nhs_no = data["pregiudicato_nhs_no"]

spacciatore_nhs_no,spacciatore_name,pregiudicato_nhs_no,pregiudicato_name
879-22-8665,Raymond,821-11-2735,Stephanie


Creo la proiezione:

In [27]:
NOME_GRAPH_DRUGS = "drugs-groups"
if graph.run("call gds.graph.exists('"+NOME_GRAPH_DRUGS+"')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("'+NOME_GRAPH_DRUGS+'")')


cq = """ 
  CALL gds.graph.project.cypher(
    '"""+NOME_GRAPH_DRUGS+"""',
    'MATCH (p) RETURN id(p) as id',
    'MATCH (a)-[r]-(b)
     RETURN id(a) as source, id(b) as target, 
      CASE
        WHEN EXISTS ( (a)-[:CURRENT_ADDRESS]-(b) ) THEN 1
        WHEN EXISTS ( (a)-[:HAS_PHONE]-(b) ) THEN 5
        WHEN EXISTS ( (a)-[:HAS_POSTCODE]-(b) ) THEN 2
        WHEN EXISTS ( (a)-[:POSTCODE_IN_AREA]-(b) ) THEN 3
        WHEN EXISTS ( (a)-[:LOCATION_IN_AREA]-(b) ) THEN 3
        WHEN EXISTS ( (a)-[:CALLER]-(b) ) THEN 1
        WHEN EXISTS ( (a)-[:CALLED]-(b) ) THEN 1
        WHEN EXISTS ( (a)-[:OCCURRED_AT]-(b) ) THEN 1
        WHEN EXISTS ( (a)-[:INVOLVED_IN]-(b) ) THEN 1
        WHEN EXISTS ( (a)-[:PARTY_TO]-(b) ) THEN 1        
        WHEN EXISTS ( (a)-[:KNOWS_LW]-(b) ) THEN 1
        WHEN EXISTS ( (a)-[:FAMILY_REL]-(b) ) THEN 2
        WHEN EXISTS ( (a)-[:KNOWS_PHONE]-(b) ) THEN 7
        WHEN EXISTS ( (a)-[:KNOWS_SN]-(b) ) THEN 9
        ELSE 1000
      END AS peso, type(r) AS type',
    {validateRelationships:false}
);  
"""
graph.run(cq)

nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,projectMillis
MATCH (p) RETURN id(p) as id,"MATCH (a)-[r]-(b) RETURN id(a) as source, id(b) as target, CASE WHEN EXISTS ( (a)-[:CURRENT_ADDRESS]-(b) ) THEN 1 WHEN EXISTS ( (a)-[:HAS_PHONE]-(b) ) THEN 5 WHEN EXISTS ( (a)-[:HAS_POSTCODE]-(b) ) THEN 2 WHEN EXISTS ( (a)-[:POSTCODE_IN_AREA]-(b) ) THEN 3 WHEN EXISTS ( (a)-[:LOCATION_IN_AREA]-(b) ) THEN 3 WHEN EXISTS ( (a)-[:CALLER]-(b) ) THEN 1 WHEN EXISTS ( (a)-[:CALLED]-(b) ) THEN 1 WHEN EXISTS ( (a)-[:OCCURRED_AT]-(b) ) THEN 1 WHEN EXISTS ( (a)-[:INVOLVED_IN]-(b) ) THEN 1 WHEN EXISTS ( (a)-[:PARTY_TO]-(b) ) THEN 1 WHEN EXISTS ( (a)-[:KNOWS_LW]-(b) ) THEN 1 WHEN EXISTS ( (a)-[:FAMILY_REL]-(b) ) THEN 2 WHEN EXISTS ( (a)-[:KNOWS_PHONE]-(b) ) THEN 7 WHEN EXISTS ( (a)-[:KNOWS_SN]-(b) ) THEN 9 ELSE 1000 END AS peso, type(r) AS type",drugs-groups,61521,211680,2840


### Memory estimation

In [28]:
print_n_results(""" 
MATCH (source:Person {nhs_no: '"""+spacciatore_nhs_no+"""'}), (target:Person {nhs_no: '"""+pregiudicato_nhs_no+"""'})
CALL gds.shortestPath.dijkstra.write.estimate('""" + NOME_GRAPH_DRUGS + """', {sourceNode: source, targetNode: target, writeRelationshipType: 'PATH'})
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN "Dijkstra senza pesi" as algoritmo, nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory

UNION ALL

MATCH (source:Person {nhs_no: '"""+spacciatore_nhs_no+"""'})
CALL gds.allShortestPaths.delta.write.estimate('""" + NOME_GRAPH_DRUGS + """', {sourceNode: source, relationshipWeightProperty: 'peso', writeRelationshipType: 'PATH' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN "All Shortest Paths con pesi" as algoritmo, nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory

""" )


algoritmo,nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
Dijkstra senza pesi,61521,211680,1984768,1984768,1938 KiB
All Shortest Paths con pesi,61521,211680,1599856,4646760,[1562 KiB ... 4537 KiB]


### Dijkstra senza pesi

In [29]:

def print_results_shortest_path(cq):
    """ Funzione che stampa i risultati dello shortest path in modo piu' leggibile """
    data = graph.run(cq).data()[0]
    print("Costo totale:", data["totalCost"])
    print("Costo ad ogni nodo:", data["costs"])
    print("Shortest Path:")

    ordine_nodi = data["ordine_nodi"]
    from pprint import pprint 
    for nodo in ordine_nodi:
        text = ""
        tipo_nodo = str(nodo.labels)[1:]
        diz = dict(nodo)
        if tipo_nodo == "Person":
            text = diz["name"]+ " " + diz["surname"] + " (" +diz["nhs_no"]+")"
        elif tipo_nodo == "Crime":
            text = "Reato del " + diz["date"] + ". Tipo: "+diz["type"]+". Esito: "+ diz["last_outcome"] 
        elif tipo_nodo == "Location" or tipo_nodo == "Area":
            text = tipo_nodo + " = " +  str(diz)[1:-1].replace("'", "", -1)
        else:
            text = nodo
        print("\t", text)

cq = """ 
MATCH (source:Person {nhs_no: '"""+spacciatore_nhs_no+"""'}), (target:Person {nhs_no: '"""+pregiudicato_nhs_no+"""'})
CALL gds.shortestPath.dijkstra.stream( '"""+NOME_GRAPH_DRUGS+ """', {sourceNode: source, targetNode: target})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
WITH totalCost, [nodeId IN nodeIds | gds.util.asNode(nodeId)] AS ordine_nodi, costs 
RETURN totalCost, ordine_nodi, costs; 
"""
print_results_shortest_path(cq)
    

Costo totale: 7.0
Costo ad ogni nodo: [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
Shortest Path:
	 Raymond Walker (879-22-8665)
	 Phillip Williamson (337-28-4424)
	 Brian Morales (335-71-7747)
	 Brenda Edwards (778-24-6852)
	 Ashley Bennett (508-10-3584)
	 Diane Bradley (338-31-6051)
	 Pamela Gibson (838-11-7607)
	 Stephanie Hughes (821-11-2735)


![Dijkstra_senza_pesi](Dijkstra_senza_pesi.png)

### All Shortest Path con pesi


In [30]:
cq = """ 
MATCH (source:Person {nhs_no: '"""+spacciatore_nhs_no+"""'}), (target:Person {nhs_no: '"""+pregiudicato_nhs_no+"""'})
CALL gds.allShortestPaths.delta.stream( '"""+NOME_GRAPH_DRUGS+ """', {sourceNode: source, relationshipWeightProperty: 'peso'})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
WITH targetNode, totalCost, [nodeId IN nodeIds | gds.util.asNode(nodeId)] AS ordine_nodi, costs
WHERE targetNode = id(target)
RETURN totalCost, ordine_nodi, costs; 
"""
# print(cq)
print_results_shortest_path(cq)

Costo totale: 12.0
Costo ad ogni nodo: [0.0, 2.0, 3.0, 4.0, 7.0, 10.0, 11.0, 12.0]
Shortest Path:
	 Raymond Walker (879-22-8665)
	 Kathleen Peters (250-75-5238)
	 Reato del 31/08/2017. Tipo: Vehicle crime. Esito: Investigation complete; no suspect identified
	 Location = postcode: BL7 9DW, address: 103 Threadfold Way, latitude: 53.614334, longitude: -2.426834
	 Area = areaCode: BL7
	 Location = postcode: BL7 9YT, address: 60 Deakins Mill Way, latitude: 53.62624, longitude: -2.442588
	 Reato del 16/08/2017. Tipo: Violence and sexual offences. Esito: Unable to prosecute suspect
	 Stephanie Hughes (821-11-2735)


![All_shortest_path_con_pesi](All_shortest_path_con_pesi.png)

### Conclusioni
...